## DS-3002: Sample Capstone Project
This notebook demonstrates many of the software libraries and programming techniques required to fulfill the requirements of the final end-of-session capstone project for course **DS-3002: Data Systems** at the University of Virginia School of Data Science. The spirit of the project is to provide a capstone challenge that requires students to demonstrate a practical and functional understanding of each of the data systems and architectural principles covered throughout the session.

**These include:**
- Relational Database Management Systems (e.g., MySQL, Microsoft SQL Server, Oracle, IBM DB2)
  - Online Transaction Processing Systems (OLTP): *Relational Databases Optimized for High-Volume Write Operations; Normalized to 3rd Normal Form.*
  - Online Analytical Processing Systems (OLAP): *Relational Databases Optimized for Read/Aggregation Operations; Dimensional Model (i.e, Star Schema)*
- NoSQL *(Not Only SQL)* Systems (e.g., MongoDB, CosmosDB, Cassandra, HBase, Redis)
- File System *(Data Lake)* Source Systems (e.g., AWS S3, Microsoft Azure Data Lake Storage)
  - Various Datafile Formats (e.g., JSON, CSV, Parquet, Text, Binary)
- Massively Parallel Processing *(MPP)* Data Integration Systems (e.g., Apache Spark, Databricks)
- Data Integration Patterns (e.g., Extract-Transform-Load, Extract-Load-Transform, Extract-Load-Transform-Load, Lambda & Kappa Architectures)

What's more, this project requires students to make effective decisions regarding whether to implement a Cloud-hosted, on-premises hosted, or hybrid architecture.

### Section I: Prerequisites

#### 1.0. Import Required Libraries

In [0]:
import os
import json
import pymongo
import pyspark.pandas as pd  # This uses Koalas that is included in PySpark version 3.2 or newer.
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BinaryType
from pyspark.sql.types import ByteType, ShortType, IntegerType, LongType, FloatType, DecimalType

#### 2.0. Instantiate Global Variables

In [0]:
# Azure SQL Server Connection Information #####################
jdbc_hostname = "laurenmds2002.mysql.database.azure.com"
jdbc_port = 3306
src_database = "police_killings"

connection_properties = {
  "user" : "lmarkwart",
  "password" : "Langley123",
  "driver" : "org.mariadb.jdbc.Driver"
}

# MongoDB Atlas Connection Information ########################
atlas_cluster_name = "Project 0"
atlas_database_name = "policekillings"
atlas_user_name = "laurenmarkwart"
atlas_password = "Password123"

# Data Files (JSON) Information ###############################
dst_database = "police_killings"

base_dir = "dbfs:/FileStore/ds3002-capstone"
database_dir = f"{base_dir}/{dst_database}"

data_dir = f"{base_dir}/source_data"
batch_dir = f"{data_dir}/batch"
stream_dir = f"{data_dir}/stream"

output_bronze = f"{database_dir}/fact_sales_orders/bronze"
output_silver = f"{database_dir}/fact_sales_orders/silver"
output_gold   = f"{database_dir}/fact_sales_orders/gold"

# Delete the Streaming Files ################################## 
dbutils.fs.rm(f"{database_dir}/fact_sales_orders", True)

# Delete the Database Files ###################################
dbutils.fs.rm(database_dir, True)

Out[3]: True

#### 3.0. Define Global Functions

In [0]:
# ######################################################################################################################
# Use this Function to Fetch a DataFrame from the Azure SQL database server.
# ######################################################################################################################
def get_sql_dataframe(host_name, port, db_name, conn_props, sql_query):
    '''Create a JDBC URL to the Azure SQL Database'''
    jdbcUrl = f"jdbc:mysql://{host_name}:{port}/{db_name}"
    
    '''Invoke the spark.read.jdbc() function to query the database, and fill a Pandas DataFrame.'''
    dframe = spark.read.jdbc(url=jdbcUrl, table=sql_query, properties=conn_props)
    
    return dframe


# ######################################################################################################################
# Use this Function to Fetch a DataFrame from the MongoDB Atlas database server Using PyMongo.
# ######################################################################################################################
def get_mongo_dataframe(user_id, pwd, cluster_name, db_name, collection, conditions, projection, sort):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.jeczt7v.mongodb.net/{db_name}?retryWrites=true&w=majority"
    
    client = pymongo.MongoClient(mongo_uri)

    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    if conditions and projection and sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection).sort(sort)))
    elif conditions and projection and not sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection)))
    else:
        dframe = pd.DataFrame(list(db[collection].find()))

    client.close()
    
    return dframe

# ######################################################################################################################
# Use this Function to Create New Collections by Uploading JSON file(s) to the MongoDB Atlas server.
# ######################################################################################################################
def set_mongo_collection(user_id, pwd, cluster_name, db_name, src_file_path, json_files):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.jeczt7v.mongodb.net/{db_name}?retryWrites=true&w=majority"
    client = pymongo.MongoClient(mongo_uri)
    db = client[db_name]
    
    '''Read in a JSON file, and Use It to Create a New Collection'''
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(src_file_path, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)

    client.close()
    
    return result

### Section II: Populate Dimensions by Ingesting Reference (Cold-path) Data 
#### 1.0. Fetch Reference Data From an Azure SQL Database
##### 1.1. Create a New Databricks Metadata Database, and then Create a New Table that Sources its Data from a View in an Azure SQL database.

In [0]:
%sql
DROP DATABASE IF EXISTS police_killings CASCADE;

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS police_killings
COMMENT "Capstone Project Database"
LOCATION "dbfs:/FileStore/ds3002-capstone/police_killings"
WITH DBPROPERTIES (contains_pii = true, purpose = "DS-3002 Capstone Project");

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW view_victim
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://laurenmds2002.mysql.database.azure.com:3306/police_killings",
  dbtable "police_killings.victim",
  user "lmarkwart",
  password "Langley123"
)

In [0]:
%sql
USE DATABASE police_killings;

CREATE TABLE IF NOT EXISTS police_killings.victim
COMMENT "Date Dimension Table"
LOCATION "dbfs:/FileStore/ds3002-capstone/police_killings/date"
AS SELECT * FROM view_victim

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM police_killings.victim LIMIT 5

victim_ID,name,age,gender,race
30,A'donte Washington,16,Male,Black
31,Aaron Rutledge,27,Male,White
32,Aaron Siler,26,Male,White
33,Aaron Valdez,25,Male,Hispanic/Latino
34,Adam Jovicic,29,Male,White


In [0]:
%sql
DESCRIBE EXTENDED police_killings.victim;

col_name,data_type,comment
victim_ID,int,
name,string,
age,string,
gender,string,
race,string,
,,
# Partitioning,,
Not partitioned,,
,,
# Detailed Table Information,,


##### 1.2. Create a New Table that Sources its Data from a Table in an Azure SQL database.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW view_location
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://laurenmds2002.mysql.database.azure.com:3306/police_killings",
  dbtable "police_killings.location",
  user "lmarkwart",
  password "Langley123"
)

In [0]:
%sql
USE DATABASE police_killings;

CREATE TABLE IF NOT EXISTS police_killings.location
COMMENT "Date Dimension Table"
LOCATION "dbfs:/FileStore/ds3002-capstone/police_killings/location"
AS SELECT * FROM view_location

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM police_killings.location LIMIT 5

location_ID,street_address,city,state,latitude,longitude
30,Clearview Ln,Millbrook,AL,32.529577,-86.362829
31,300 block Iris Park Dr,Pineville,LA,31.3217392,-92.4348602
32,22nd Ave and 56th St,Kenosha,WI,42.5835597,-87.8357101
33,3000 Seminole Ave,South Gate,CA,33.9392976,-118.2194634
34,364 Hiwood Ave,Munroe Falls,OH,41.1485748,-81.4298782


In [0]:
%sql
DESCRIBE EXTENDED police_killings.location;

col_name,data_type,comment
location_ID,int,
street_address,string,
city,string,
state,string,
latitude,string,
longitude,string,
,,
# Partitioning,,
Not partitioned,,
,,


#### 2.0. Fetch Reference Data from a MongoDB Atlas Database
##### 2.1. View the Data Files on the Databricks File System

In [0]:
display(dbutils.fs.ls(batch_dir))

##### 2.2. Create a New MongoDB Database, and Load JSON Data Into a New MongoDB Collection
**NOTE:** The following cell **can** be run more than once because the **set_mongo_collection()** function **is** idempotent.

In [0]:
source_dir = '/dbfs/FileStore/ds3002-capstone/source_data/batch'
json_files = {"population_facts" : 'population_facts.json'}

set_mongo_collection(atlas_user_name, atlas_password, atlas_cluster_name, atlas_database_name, source_dir, json_files) 

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-715934921754231> in <module> 
 2 json_files = { "population_facts" : 'population_facts.json' } 
 3 
 ----> 4 set_mongo_collection ( atlas_user_name , atlas_password , atlas_cluster_name , atlas_database_name , source_dir , json_files ) 

 NameError : name 'set_mongo_collection' is not defined

##### 2.3. Fetch Data from the New MongoDB Collection

In [0]:
%scala
import com.mongodb.spark._

val df_population_facts = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("database", "policekillings").option("collection", "population_facts").load()
display(df_population_facts)

at com.mongodb.internal.connection.SaslAuthenticator.wrapException(SaslAuthenticator.java:201)
	at com.mongodb.internal.connection.SaslAuthenticator.access$300(SaslAuthenticator.java:40)
	at com.mongodb.internal.connection.SaslAuthenticator$1.run(SaslAuthenticator.java:78)
	at com.mongodb.internal.connection.SaslAuthenticator$1.run(SaslAuthenticator.java:47)
	at com.mongodb.internal.connection.SaslAuthenticator.doAsSubject(SaslAuthenticator.java:207)
	at com.mongodb.internal.connection.SaslAuthenticator.authenticate(SaslAuthenticator.java:47)
	at com.mongodb.internal.connection.InternalStreamConnectionInitializer.authenticate(InternalStreamConnectionInitializer.java:152)
	at com.mongodb.internal.connection.InternalStreamConnectionInitializer.initialize(InternalStreamConnectionInitializer.java:61)
	at com.mongodb.internal.connection.InternalStreamConnection.open(InternalStreamConnection.java:128)
	at com.mongodb.internal.connection.UsageTrackingInternalConnection.open(UsageTrackingInternalConnection.java:50)
	at com.mongodb.internal.connection.DefaultConnectionPool$PooledConnection.open(DefaultConnectionPool.java:435)
	at com.mongodb.internal.connection.DefaultConnectionPool.get(DefaultConnectionPool.java:117)
	at com.mongodb.internal.connection.DefaultConnectionPool.get(DefaultConnectionPool.java:102)
	at com.mongodb.internal.connection.DefaultServer.getConnection(DefaultServer.java:90)
	at com.mongodb.internal.binding.ClusterBinding$ClusterBindingConnectionSource.getConnection(ClusterBinding.java:119)
	at com.mongodb.client.internal.ClientSessionBinding$SessionBindingConnectionSource.getConnection(ClientSessionBinding.java:135)
	at com.mongodb.internal.operation.CommandOperationHelper$5.call(CommandOperationHelper.java:207)
	at com.mongodb.internal.operation.OperationHelper.withReadConnectionSource(OperationHelper.java:462)
	at com.mongodb.internal.operation.CommandOperationHelper.executeCommand(CommandOperationHelper.java:203)
	at com.mongodb.internal.operation.CommandOperationHelper.executeCommand(CommandOperationHelper.java:198)
	at com.mongodb.internal.operation.CommandReadOperation.execute(CommandReadOperation.java:58)
	at com.mongodb.client.internal.MongoClientDelegate$DelegateOperationExecutor.execute(MongoClientDelegate.java:190)
	at com.mongodb.client.internal.MongoDatabaseImpl.executeCommand(MongoDatabaseImpl.java:194)
	at com.mongodb.client.internal.MongoDatabaseImpl.runCommand(MongoDatabaseImpl.java:163)
	at com.mongodb.client.internal.MongoDatabaseImpl.runCommand(MongoDatabaseImpl.java:158)
	at com.mongodb.spark.MongoConnector.$anonfun$hasSampleAggregateOperator$1(MongoConnector.scala:234)
	at com.mongodb.spark.MongoConnector.$anonfun$withDatabaseDo$1(MongoConnector.scala:171)
	at com.mongodb.spark.MongoConnector.withMongoClientDo(MongoConnector.scala:154)
	at com.mongodb.spark.MongoConnector.withDatabaseDo(MongoConnector.scala:171)
	at com.mongodb.spark.MongoConnector.hasSampleAggregateOperator(MongoConnector.scala:234)
	at com.mongodb.spark.rdd.MongoRDD.hasSampleAggregateOperator$lzycompute(MongoRDD.scala:221)
	at com.mongodb.spark.rdd.MongoRDD.hasSampleAggregateOperator(MongoRDD.scala:221)
	at com.mongodb.spark.sql.MongoInferSchema$.apply(MongoInferSchema.scala:68)
	at com.mongodb.spark.sql.DefaultSource.constructRelation(DefaultSource.scala:97)
	at com.mongodb.spark.sql.DefaultSource.createRelation(DefaultSource.scala:50)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:385)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:356)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:323)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:323)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:222)
	at $line85caf0454a3f4526b9351b81c2889f0125.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-715934921754233:3)
	at $line85caf0454a3f4526b9351b81c2889

In [0]:
%scala
df_population_facts.printSchema()

##### 2.4. Use the Spark DataFrame to Create a New Table in the Databricks (Adventure Works) Metadata Database

In [0]:
%scala
df_population_facts.write.format("delta").mode("overwrite").saveAsTable("police_killings.population_facts")

In [0]:
%sql
DESCRIBE EXTENDED police_killings.population_facts

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-715934921754237> in <module> 
 5 display ( df ) 
 6 return df
 ----> 7 _sqldf = ____databricks_percent_sql ( ) 
 8 finally : 
 9 del ____databricks_percent_sql

 <command-715934921754237> in ____databricks_percent_sql () 
 2 def ____databricks_percent_sql ( ) : 
 3 import base64
 ----> 4 df = spark . sql ( base64 . standard_b64decode ( "REVTQ1JJQkUgRVhURU5ERUQgcG9saWNlX2tpbGxpbmdzLnBvcHVsYXRpb25fZmFjdHM=" ) . decode ( ) ) 
 5 display ( df ) 
 6 return df

 /databricks/spark/python/pyspark/sql/session.py in sql (self, sqlQuery) 
 775 [ Row ( f1 = 1 , f2 = 'row1' ) , Row ( f1 = 2 , f2 = 'row2' ) , Row ( f1 = 3 , f2 = 'row3' ) ] 
 776 """
 --> 777 return DataFrame ( self . _jsparkSession . sql ( sqlQuery ) , self . _wrapped ) 
 778 
 779 def table ( self , tableName ) : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Table or view not found: police_killings.population_facts; line 1 pos 18;
'DescribeRelation true, [col_name#59604, data_type#59605, comment#59606]
+- 'UnresolvedTableOrView [police_killings, population_facts], DESCRIBE TABLE, true

##### 2.5. Query the New Table in the Databricks Metadata Database

In [0]:
%sql
SELECT * FROM police_killings.population_facts LIMIT 5

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-715934921754239> in <module> 
 5 display ( df ) 
 6 return df
 ----> 7 _sqldf = ____databricks_percent_sql ( ) 
 8 finally : 
 9 del ____databricks_percent_sql

 <command-715934921754239> in ____databricks_percent_sql () 
 2 def ____databricks_percent_sql ( ) : 
 3 import base64
 ----> 4 df = spark . sql ( base64 . standard_b64decode ( "U0VMRUNUICogRlJPTSBwb2xpY2Vfa2lsbGluZ3MudmljdGltIExJTUlUIDU=" ) . decode ( ) ) 
 5 display ( df ) 
 6 return df

 /databricks/spark/python/pyspark/sql/session.py in sql (self, sqlQuery) 
 775 [ Row ( f1 = 1 , f2 = 'row1' ) , Row ( f1 = 2 , f2 = 'row2' ) , Row ( f1 = 3 , f2 = 'row3' ) ] 
 776 """
 --> 777 return DataFrame ( self . _jsparkSession . sql ( sqlQuery ) , self . _wrapped ) 
 778 
 779 def table ( self , tableName ) : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Table or view not found: police_killings.victim; line 1 pos 14;
'GlobalLimit 5
+- 'LocalLimit 5
 +- 'Project [*]
 +- 'UnresolvedRelation [police_killings, victim], [], false

#### 3.0. Fetch Data from a File System
##### 3.1. Use PySpark to Read From a CSV File

In [0]:
cause_csv = f"{batch_dir}/cause.csv"

df_incident_facts = spark.read.format('csv').options(header='true', inferSchema='true').load(cause_csv)
display(df_incident_facts)

law_enforcement_agency,cause,armed
Millbrook Police Department,Gunshot,No
Rapides Parish Sheriff's Office,Gunshot,No
Kenosha Police Department,Gunshot,No
South Gate Police Department,Gunshot,Firearm
Kent Police Department,Gunshot,No
Phoenix Police Department,Gunshot,No
Bakersfield Police Department,Gunshot,Firearm
Los Angeles Police Department,Gunshot,Non-lethal firearm
Wise County Sheriff's Department and Texas DPS,Gunshot,Firearm
Kentwood Police Department and Wyoming DPS,Gunshot,Other


In [0]:
df_incident_facts.printSchema()

root
-- law_enforcement_agency: string (nullable = true)
-- cause: string (nullable = true)
-- armed: string (nullable = true)

In [0]:
df_incident_facts.write.format("delta").mode("overwrite").saveAsTable("police_killings.incident_facts")

In [0]:
%sql
DESCRIBE EXTENDED police_killings.incident_facts;

col_name,data_type,comment
law_enforcement_agency,string,
cause,string,
armed,string,
,,
# Partitioning,,
Not partitioned,,
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,police_killings,


In [0]:
%sql
SELECT * FROM police_killings.incident_facts LIMIT 5;

law_enforcement_agency,cause,armed
Millbrook Police Department,Gunshot,No
Rapides Parish Sheriff's Office,Gunshot,No
Kenosha Police Department,Gunshot,No
South Gate Police Department,Gunshot,Firearm
Kent Police Department,Gunshot,No


##### Verify Dimension Tables

In [0]:
%sql
USE police_killings;
SHOW TABLES

database,tableName,isTemporary
police_killings,incident_facts,false
police_killings,location,false
police_killings,victim,false
,view_date,true
,view_location,true
,view_victim,true


### Section III: Integrate Reference Data with Real-Time Data
#### 6.0. Use AutoLoader to Process Streaming (Hot Path) Data 
##### 6.1. Bronze Table: Process 'Raw' JSON Data

In [0]:
(spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "json")
 .option("cloudFiles.schemaHints", "Incident_ID INT")
 .option("cloudFiles.schemaHints", "law_enforcement_agency STRING")
 .option("cloudFiles.schemaHints", "cause STRING") 
 .option("cloudFiles.schemaHints", "armed STRING")
 .option("cloudFiles.schemaHints", "location_ID INT")
 .option("cloudFiles.schemaHints", "street_address STRING")
 .option("cloudFiles.schemaHints", "city STRING")
 .option("cloudFiles.schemaHints", "state STRING")
 .option("cloudFiles.schemaHints", "victim_ID STRING")
 .option("cloudFiles.schemaHints", "name STRING")
 .option("cloudFiles.schemaHints", "age STRING")
 .option("cloudFiles.schemaHints", "gender STRING")
 .option("cloudFiles.schemaHints", "race STRING")
 .option("cloudFiles.schemaLocation", output_bronze)
 .option("cloudFiles.inferColumnTypes", "true")
 .option("multiLine", "true")
 .load(stream_dir)
 .createOrReplaceTempView("orders_raw_tempview"))

In [0]:
%sql
/* Add Metadata for Traceability */
CREATE OR REPLACE TEMPORARY VIEW orders_bronze_tempview AS (
  SELECT *, current_timestamp() receipt_time, input_file_name() source_file
  FROM orders_raw_tempview
)

In [0]:
%sql
SELECT * FROM orders_bronze_tempview

age,armed,cause,city,college_degree_rate,gender,latitude,law_enforcement_agency,longitude,median_household_income,median_personal_income,name,population_size,poverty_rate,race,share_black,share_hispanic,share_white,state,street_address,unemployment_rate,_rescued_data,receipt_time,source_file
null,null,null,null,0.168509509,null,null,null,null,51367,28375,null,3779,14.1,null,30.5,5.6,60.5,null,null,0.097686375,null,2022-12-14T05:20:59.828+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.111402359,null,null,null,null,27972,14678,null,2769,28.8,null,36.2,0.5,53.8,null,null,0.065723794,null,2022-12-14T05:20:59.828+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.147312269,null,null,null,null,45365,25286,null,4079,14.6,null,7.7,16.8,73.8,null,null,0.166293142,null,2022-12-14T05:20:59.828+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.050132928,null,null,null,null,48295,17194,null,4343,11.7,null,0.6,98.8,1.2,null,null,0.124827269,null,2022-12-14T05:20:59.828+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.403954214,null,null,null,null,68785,33954,null,6809,1.9,null,1.4,1.7,92.5,null,null,0.063549832,null,2022-12-14T05:20:59.828+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.102955195,null,null,null,null,20833,15523,null,4682,58,null,7.7,79,7,null,null,0.073651452,null,2022-12-14T05:20:59.828+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.20380117,null,null,null,null,58068,25949,null,5027,17.2,null,0.3,44.2,50.8,null,null,0.131461131,null,2022-12-14T05:20:59.828+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.090437601,null,null,null,null,66543,25043,null,5238,12.2,null,0.2,84.1,8.6,null,null,0.094346979,null,2022-12-14T05:20:59.828+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.047601339,null,null,null,null,30391,16778,null,4832,37.7,null,17.7,66.3,14.6,null,null,0.140832976,null,2022-12-14T05:20:59.828+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.102691511,null,null,null,null,44553,22005,null,3795,18.4,null,7.7,26.5,63.6,null,null,0.174167417,null,2022-12-14T05:20:59.828+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json


In [0]:
(spark.table("orders_bronze_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{output_bronze}/_checkpoint")
      .outputMode("append")
      .table("fact_orders_bronze"))

Out[13]: <pyspark.sql.streaming.StreamingQuery at 0x7f9f05fd56a0>

##### 6.2. Silver Table: Include Reference Data

In [0]:
(spark.readStream
  .table("fact_orders_bronze")
  .createOrReplaceTempView("orders_silver_tempview"))

In [0]:
%sql
SELECT * FROM orders_silver_tempview

age,armed,cause,city,college_degree_rate,gender,latitude,law_enforcement_agency,longitude,median_household_income,median_personal_income,name,population_size,poverty_rate,race,share_black,share_hispanic,share_white,state,street_address,unemployment_rate,_rescued_data,receipt_time,source_file
null,null,null,null,0.168509509,null,null,null,null,51367,28375,null,3779,14.1,null,30.5,5.6,60.5,null,null,0.097686375,null,2022-12-14T05:23:26.001+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.111402359,null,null,null,null,27972,14678,null,2769,28.8,null,36.2,0.5,53.8,null,null,0.065723794,null,2022-12-14T05:23:26.001+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.147312269,null,null,null,null,45365,25286,null,4079,14.6,null,7.7,16.8,73.8,null,null,0.166293142,null,2022-12-14T05:23:26.001+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.050132928,null,null,null,null,48295,17194,null,4343,11.7,null,0.6,98.8,1.2,null,null,0.124827269,null,2022-12-14T05:23:26.001+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.403954214,null,null,null,null,68785,33954,null,6809,1.9,null,1.4,1.7,92.5,null,null,0.063549832,null,2022-12-14T05:23:26.001+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.102955195,null,null,null,null,20833,15523,null,4682,58,null,7.7,79,7,null,null,0.073651452,null,2022-12-14T05:23:26.001+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.20380117,null,null,null,null,58068,25949,null,5027,17.2,null,0.3,44.2,50.8,null,null,0.131461131,null,2022-12-14T05:23:26.001+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.090437601,null,null,null,null,66543,25043,null,5238,12.2,null,0.2,84.1,8.6,null,null,0.094346979,null,2022-12-14T05:23:26.001+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.047601339,null,null,null,null,30391,16778,null,4832,37.7,null,17.7,66.3,14.6,null,null,0.140832976,null,2022-12-14T05:23:26.001+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json
null,null,null,null,0.102691511,null,null,null,null,44553,22005,null,3795,18.4,null,7.7,26.5,63.6,null,null,0.174167417,null,2022-12-14T05:23:26.001+0000,dbfs:/FileStore/ds3002-capstone/source_data/stream/population.json


In [0]:
%sql
DESCRIBE EXTENDED orders_silver_tempview

col_name,data_type,comment
age,string,null
armed,string,null
cause,string,null
city,string,null
college_degree_rate,string,null
gender,string,null
latitude,double,null
law_enforcement_agency,string,null
longitude,double,null
median_household_income,string,null


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fact_orders_silver_tempview AS (
  SELECT victim.Victim_ID
    , victim.name AS VictimName
    , victim.age AS VictimAge
    , victim.race AS VictimRace
    , population_facts.population_size
    , population_facts.share_white
    , population_facts.share_black
    , population_facts.share_hispanice
    , population_facts.poverty_rate
    , population_facts.unemployment_rate
    , population_facts.college_degree_rate
    , population_facts.median_personal_income
    , incident_facts.law_enforcement_agency
    , incident_facts.armed
    , incident_facts.cause
    , location.location_ID
    , location.street_address
    , location.city
    , location.state
    , t.Incident_ID
    , t.location_ID
    , t.victim_ID
    , t.population_facts_ID
  FROM orders_silver_tempview t
  INNER JOIN police_killings.victim c
  ON t.Incident_ID = c.victim_ID
  INNER JOIN police_killings.location sa
  ON t.population_facts_ID = CAST(sa.location_ID AS BIGINT)

In [0]:
(spark.table("fact_orders_silver_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{output_silver}/_checkpoint")
      .outputMode("append")
      .table("fact_orders_silver"))

In [0]:
%sql
SELECT * FROM fact_orders_silver

In [0]:
%sql
DESCRIBE EXTENDED police_killings.fact_orders_silver

##### 6.4. Gold Table: Perform Aggregations

In [0]:
%sql
SELECT victim_ID
  , VictimName
  , VictimAge
  , VictimRace
  , COUNT(victim_ID) AS victimID
FROM police_killings.fact_orders_silver
GROUP BY Victim_ID, VictimAge, VictimRace, VictimName
ORDER BY victimID DESC

In [0]:
%sql
SELECT pc.incident_facts
  , os.cause AS CauseOfDeath
  , os.armed
  , pc.poverty_rate
FROM police_killings.fact_orders_silver AS os
INNER JOIN (
  SELECT Incident_ID
  , COUNT(Victim_ID) AS VictimID
  FROM police_killings.fact_orders_silver
  GROUP BY population_facts_ID
) AS pc
ON pc.population_facts_ID = os.Incident_facts_ID
ORDER BY Population_Facts_ID DESC